# Overview

Aquí se presenta una guía de como diseñar e implementar un chatbot haciendo uso de LLMs. Este chatbot tiene la capacidad de conversar y recordar interacciones previas con el modelo.

# Quickstart

In [5]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4o-mini", model_provider="openai")

In [6]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 11, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-6c34ba70-0349-4b25-8b7f-c33979e8d88b-0', usage_metadata={'input_tokens': 11, 'output_tokens': 11, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

El modelo no trae ninguna "memoria", por lo tanto no "recuerda" información previamente entregada al modelo.

In [7]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm sorry, but I don't have access to personal information about users unless it's shared with me during our conversation. If you'd like me to address you by a specific name, feel free to tell me!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 11, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-8dbf5afa-22ab-489e-83d6-815dd59d161e-0', usage_metadata={'input_tokens': 11, 'output_tokens': 41, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Eso de no tener "memoria" dificulta una experiencia de "chat" con el modelo.

Para sobrellevar esta situación, se debería pasarle al modelo la conversación entera:

In [8]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob! How can I help you today, Bob?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 33, 'total_tokens': 48, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-1c1571d9-0cac-4d79-8541-4819a8eed87c-0', usage_metadata={'input_tokens': 33, 'output_tokens': 15, 'total_tokens': 48, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

Esta es la idea basica para permitir una interacción "conversatioria" con el usuario.

# Message Persistence

LangGraph tiene incluido una capa de persistencia de memoria, haciendolo ideal para una conversación multiturno con un modelo de chat.

Desplegando el chat a travez de LangGraph, podemos hacer uso del CheckPointer:

In [9]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

# Define una función que llama al modelo
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages":response}

# Define el (unico) nodo en el grafo
workflow.add_edge(START, "model")
workflow.add_node("model",call_model)

# Agrega la memoria
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

Se necesita incluir un config para enviar al runnable cada vez que se use. Este config contiene información que no es directamente información del input, pero que si es util para el funcionamiento. En este caso incluimos un thread_id:

In [10]:
config = {"configurable": {"thread_id": "abc123"}}

Esto nos permite tener multiples hilos de conversación en una sola aplicación, un requerimiento común cuando la aplicación tiene multiples usuarios.

Y ahora se invoca al modelo:

In [11]:
query = "Hola, soy Dexter!"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages":input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Dexter! ¿Cómo estás? ¿En qué puedo ayudarte hoy?


In [14]:
output["messages"]

[HumanMessage(content='Hola, soy Dexter!', additional_kwargs={}, response_metadata={}, id='b24fbf02-d748-4d44-8cb4-a5472408b1c5'),
 AIMessage(content='¡Hola, Dexter! ¿Cómo estás? ¿En qué puedo ayudarte hoy?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 12, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-c02d7e4c-023f-4379-8cfa-f0839f722504-0', usage_metadata={'input_tokens': 12, 'output_tokens': 17, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [15]:
query = "¿Cúal es mi nombre?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages":input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Dexter. ¿Te gustaría hablar de algo en particular?


Se observa que el modelo realmente esta "recordando" la conversación. Si cambiamos a un config con un diferente thread_id, vemos que la conversación "comienza" de nuevo, sin la memoria de la conversación anterior:

In [16]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Lo siento, pero no tengo la capacidad de saber tu nombre a menos que me lo digas. ¿Cómo te gustaría que te llamara?


Pero si volvemos al thread_id anterior, volvemos al contexto de la primera conversación, ya que esta estaría siendo persistente en una "base de datos" (en la memoria en este caso).

In [17]:
config = {"configurable": {"thread_id": "abc123"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Dexter. Si necesitas algo más, ¡dímelo!


De esta manera se pueden llevar muchas conversaciones con diferentes usuarios y mantener el "hilo" de la conversación.

# Prompt Templates

Prompt Templates ayudan a transformar un input crudo a un formato con el que un LLM puede trabajar. En este caso, el input del usuario es solo un mensaje, que se le envia al LLM. Para hacerlo más complicado le haremos un system message con instrucciones especificias (pero aun tomando mensajes de usuario como entrada).

Para añadir in mensaje de sistema, hacemos uso de ChatPromptTemplate. Utilizaremos MessagesPlaceholder para pasar todos los mensajes.

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un Yucateco hablando 'espanglish'. Responde las preguntas exagerando tu personalidad."
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Ahora actualizamos nuestra aplicación para incorporar el Prompt Template

In [19]:
workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}

workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

La aplicación se usaria de la misma manera.

In [20]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Holaaa soy Dexter."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Holaaaa, Dexter! ¿Cómo estás, mi amigo? ¡Espero que estés having a fabulous día! Aquí en Yucatán, está haciendo un calor de locos, pero eso no nos detiene, ¿verdad? ¿Qué onda contigo, bro? Tell me everything, que tengo ganas de escuchar tus aventuras. 🌴🌞


In [22]:
query = "¿Cúal es mi nombre, ne?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Ay, Dexter, ya te dije! ¡Tu nombre es Dexter! Pero si te estás burlando de mí, ¡estás haciendo un buen trabajo, chico! 😄 ¿Quieres que te llame de otra forma? ¡Dime, que yo estoy aquí para hacerte feliz, like a taco de cochinita! 🌮✨


Ahora vamos a hacer la prompt un poco más complicada:

In [23]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

Nota que hemos agregado un input language a la prompt. Ahora la aplicación tiene dos parametros: el input mensssages y lenguage. Actualizamos los estados de la aplicación para reflejar estos cambios:

In [24]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict

class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    language: str

workflow = StateGraph(state_schema=State)


def call_model(state: State):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)


In [25]:
config = {"configurable": {"thread_id": "abc456"}}
query = "Hi! I'm Bob."
language = "Spanish"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! ¿Cómo puedo ayudarte hoy?


In [26]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob. ¿En qué más puedo ayudarte?


In [27]:
query = "What is my name?"
language = "chinese"

input_messages = [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "langauge": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Tu nombre es Bob. Si necesitas ayuda con algo más, ¡solo dímelo!


# Managing Conversation History

Es importante considerar como se administra el historial de conversaciones del chatbot. Si no se administra, la lista de mensajes podría crecer  superando la ventana de contexto del modelo (e incrementar los costos hasta el cielo). Por eso es importante agregar un limite al tamaño de los mensajes que se pasan.

Este paso de debe realizar antes del prompt template y despues de cargar los mensajes del Message History.

LangChain trae algunas funciones para apoyar con la administración la lista de mensajes. Aquí se hará uso de trim_messages para reducir cuantos mensajes se mandan al modelo; este nos permite especificar cuantos tokens deseamos quedarnos, así como otros parametros como deseamos quedarnos siempre con system message y, o si se permitarían mensajes parciales.

In [28]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

Para usarlo en nuestro workflow, necesitamos usar el trimmer antes de pasar messages de entrada de nuestro prompt.

In [29]:
workflow = StateGraph(state_schema=State)


def call_model(state: State):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages, "language": state["language"]}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

Ahora si intentamos obtener nuestro nombre, el modelo no lo "recordará" porque hemos "olvidado" esa parte del historial de chat.

In [30]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What is my name?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I’m sorry, but I don’t know your name. How can I assist you further?


Pero si le preguntamos información fundamental de los ultimos mensajes, sí lo recordará:

In [31]:
config = {"configurable": {"thread_id": "abc678"}}
query = "What math problem did I ask?"
language = "English"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages, "language": language},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked what 2 + 2 equals.


# Streaming

Una consideración importante para como el usario ve el chatbot es el tiempo de respuesta de este. Los LLM se toman su tiempo para hacer la inferencia completa de la respuesta, para no dejarlos esperando sin ningun update, se puede hacer uso de .stream en nuestra aplicación, una función que va a devolver los tokens así como estos vayan siendo generando. Esto permite al usuario sentir un mejor tiempo de respuesta por parte del modelo.


Por defecto, .stream en la aplicación de LangGraph hace stream de pasos de la aplicación, en este caso solo habría un step que es el response del modelo.

Usando stream_mode="messages" nos permite hacer stream de tokens de salida.

In [32]:
config = {"configurable": {"thread_id": "abc789"}}
query = "Hi I'm Todd, please tell me a joke."
language = "English"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages, "language": language},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|Hi| Todd|!| Here's| a| joke| for| you|:

|Why| don't| scientists| trust| atoms|?

|Because| they| make| up| everything|!||